In [0]:
sc

SparkContext 

 Spark UI 

 
 Version 
 v3.2.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
print(spark)

In [0]:
# a)
flights_delay=spark.read.csv("dbfs:/FileStore/tables/Flights_Delay.csv",header= True ,inferSchema=True)

In [0]:
#Create Airline Pysaprk sql dataFrame. 
flights_delay.show()

+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+------------+--------+--------+---------+-------+-----------------+------------+-------------+--------+---------+-------------------+----------------+--------------+-------------+-------------------+-------------+
| ID|YEAR|MONTH|DAY|DAY_OF_WEEK|AIRLINE|FLIGHT_NUMBER|TAIL_NUMBER|ORIGIN_AIRPORT|DESTINATION_AIRPORT|SCHEDULED_DEPARTURE|DEPARTURE_TIME|DEPARTURE_DELAY|TAXI_OUT|WHEELS_OFF|SCHEDULED_TIME|ELAPSED_TIME|AIR_TIME|DISTANCE|WHEELS_ON|TAXI_IN|SCHEDULED_ARRIVAL|ARRIVAL_TIME|ARRIVAL_DELAY|DIVERTED|CANCELLED|CANCELLATION_REASON|AIR_SYSTEM_DELAY|SECURITY_DELAY|AIRLINE_DELAY|LATE_AIRCRAFT_DELAY|WEATHER_DELAY|
+---+----+-----+---+-----------+-------+-------------+-----------+--------------+-------------------+-------------------+--------------+---------------+--------+----------+--------------+-----------

In [0]:
#b)	Describe the table schema & show 10 rows of Dataset
flights_delay.printSchema()
flights_delay.show(10)

root
 |-- ID: integer (nullable = true)
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true

In [0]:
#c)	Average arrival delay caused by airlines.
flights_delay.groupBy('FLIGHT_NUMBER').mean('ARRIVAL_DELAY').show()

+-------------+-------------------+
|FLIGHT_NUMBER| avg(ARRIVAL_DELAY)|
+-------------+-------------------+
|         1088|  10.68421052631579|
|         1238|              -3.35|
|         2866|           -16.4375|
|         4935|                7.0|
|          148| 0.4444444444444444|
|         4101|0.16666666666666666|
|         5803|               -3.8|
|         6397|               -8.0|
|          833| 10.212121212121213|
|         2142| 0.6363636363636364|
|         4519|               11.0|
|         1591|               -1.0|
|         4900|            -2.4375|
|         2659|               -2.0|
|         1342|               -9.8|
|         1959|                5.0|
|         5518|               12.8|
|         4818| 12.666666666666666|
|         6466| 12.571428571428571|
|         5300|               47.0|
+-------------+-------------------+
only showing top 20 rows



In [0]:
#d)	Days of months with average Arrival delays.
flights_delay.groupBy('DAY').mean('ARRIVAL_DELAY').show()

+---+------------------+
|DAY|avg(ARRIVAL_DELAY)|
+---+------------------+
| 31|-1.196594427244582|
| 28| 3.257425742574257|
| 27| 4.706711409395973|
| 26| 11.96778269109286|
| 12| 11.24892703862661|
| 22| 6.550920245398773|
|  1|14.807807807807809|
| 13|3.3769751693002257|
|  6|10.608832807570979|
| 16| 9.124321062160531|
|  3|18.141541038525965|
| 20|3.8770149253731345|
|  5| 16.23861262014208|
| 19|1.6344282238442822|
| 15| 2.966753585397653|
|  9| 4.421887390959556|
| 17| 8.761435608726249|
|  4|17.157790927021697|
|  8| 5.232349165596919|
| 23| 4.207086133170434|
+---+------------------+
only showing top 20 rows



In [0]:
#f)	Arrange weekdays with respect to the average arrival delays
flights_delay.groupBy('DAY_OF_WEEK').mean('ARRIVAL_DELAY').show()

+-----------+------------------+
|DAY_OF_WEEK|avg(ARRIVAL_DELAY)|
+-----------+------------------+
|          1|10.807447207297264|
|          6| 4.888689138576779|
|          3| 5.587079407806191|
|          5| 6.010538373424971|
|          4| 7.174969021065675|
|          7|10.110840438489646|
|          2| 8.033644102148358|
+-----------+------------------+



In [0]:
flights_delay.createOrReplaceTempView("datatable")

In [0]:
#e)	Days of months with median Departure delays.
spark.sql("select DAY, PERCENTILE_APPROX(DEPARTURE_DELAY, 0.5) as medianDelay from datatable group by DAY order by medianDelay desc").show()

+---+-----------+
|DAY|medianDelay|
+---+-----------+
|  2|          2|
|  5|          1|
|  3|          1|
| 12|          0|
|  6|          0|
| 17|          0|
|  1|          0|
|  4|          0|
| 16|         -1|
| 15|         -1|
|  9|         -1|
| 23|         -1|
| 26|         -1|
|  8|         -1|
| 22|         -1|
| 13|         -1|
| 28|         -2|
| 27|         -2|
| 20|         -2|
| 19|         -2|
+---+-----------+
only showing top 20 rows



In [0]:
#g)	Show Analysis for each month with total number of cancellations. (Descending).
spark.sql("select MONTH, sum(CANCELLED) as TOTALCANCELLED from datatable group by MONTH order by TOTALCANCELLED desc").show()

+-----+--------------+
|MONTH|TOTALCANCELLED|
+-----+--------------+
|    2|          1087|
|    1|           608|
|    3|           456|
+-----+--------------+



In [0]:
#i)	Show the airlines that make the maximum number of cancellations.
flights_delay=spark.sql("select AIRLINE,count(*) as CANCELLED from datatable group by AIRLINE order by CANCELLED desc")
flights_delay.show()

+-------+---------+
|AIRLINE|CANCELLED|
+-------+---------+
|     WN|    11738|
|     DL|     7989|
|     EV|     5916|
|     OO|     5708|
|     AA|     5250|
|     UA|     4701|
|     US|     3925|
|     MQ|     3502|
|     B6|     2548|
|     AS|     1586|
|     NK|     1048|
|     F9|      794|
|     HA|      722|
|     VX|      573|
+-------+---------+



In [0]:
#j)	Find and order airlines in descending that make the most number of diversions.
flights_delay=spark.sql("select AIRLINE,count(*) as DIVERTED from datatable group by AIRLINE order by DIVERTED desc")
flights_delay.show()

+-------+--------+
|AIRLINE|DIVERTED|
+-------+--------+
|     WN|   11738|
|     DL|    7989|
|     EV|    5916|
|     OO|    5708|
|     AA|    5250|
|     UA|    4701|
|     US|    3925|
|     MQ|    3502|
|     B6|    2548|
|     AS|    1586|
|     NK|    1048|
|     F9|     794|
|     HA|     722|
|     VX|     573|
+-------+--------+



In [0]:
#n)	Find all diverted Route from a source to destination Airport & which route is the most diverted.
flights_delay=spark.sql("select AIRLINE,count(*) as DIVERTED from datatable group by AIRLINE order by DIVERTED desc")
flights_delay.show()

+-------+--------+
|AIRLINE|DIVERTED|
+-------+--------+
|     WN|   11738|
|     DL|    7989|
|     EV|    5916|
|     OO|    5708|
|     AA|    5250|
|     UA|    4701|
|     US|    3925|
|     MQ|    3502|
|     B6|    2548|
|     AS|    1586|
|     NK|    1048|
|     F9|     794|
|     HA|     722|
|     VX|     573|
+-------+--------+

